## Demo for the Calculation of the Semantic Brand Score - Basic Version

In [1]:
# Read text documents from an example CSV file
import csv
readfile = csv.reader(open("AliceWonderland.csv", 'rt',  encoding="utf8"), delimiter = "|", quoting=csv.QUOTE_NONE)
texts = [line[0] for line in readfile]
#4 Chapters of Alice in Wonderland
print(len(texts))
print(texts[0][:200])

4
Down the Rabbit-Hole. Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it


I <font color="#d81e5b">**imported the text file**</font> in Python as a list of text documents (texts), which are processed to remove punctuation, stop-words and special characters. Words are lowercased and split into tokens, thus obtaining a new texts variable, which is a list of lists. More complex operations of <font color="#d81e5b">**text preprocessing**</font> are always possible (such as the removal of html tags or ‘#’), for which I recommend reading one of many tutorials on Natural Language Processing in Python. The stopwords list is taken from the NLTK package. Lastly, word affixes are remove through Snowball Stemming (lemming or other approaches would be possible, also depending on the language of texts).

In [2]:
##Import re, string and nltk, and download stop-words
import re
import nltk
import string
from nltk.stem.snowball import SnowballStemmer

#Define stopwords
#nltk.download("stopwords")
stopw = nltk.corpus.stopwords.words('english') + ['‘']

#Define brands (lowercase)
brands = ['alice', 'rabbit']

# texts is a list of strings, one for each document analyzed.

#Convert to lowercase
texts = [t.lower() for t in texts]
#Remove words that start with HTTP
texts = [re.sub(r"http\S+", " ", t) for t in texts]
#Remove words that start with WWW
texts = [re.sub(r"www\S+", " ", t) for t in texts]
#Remove punctuation
regex = re.compile('[%s]' % re.escape(string.punctuation))
texts = [regex.sub(' ', t) for t in texts]
#Remove words made of single letters
texts = [re.sub(r'\b\w{1}\b', ' ', t) for t in texts]
#Remove stopwords
pattern = re.compile(r'\b(' + r'|'.join(stopw) + r')\b\s*')
texts = [pattern.sub(' ', t) for t in texts]
#Remove additional whitespaces
texts = [re.sub(' +',' ',t) for t in texts]

#Tokenize text documents (becomes a list of lists)
texts = [t.split() for t in texts]

# Snowball Stemming
stemmer = SnowballStemmer("english")
texts = [[stemmer.stem(w) if w not in brands else w for w in t] for t in texts]
texts[0][:6]

['rabbit', 'hole', 'alice', 'begin', 'get', 'tire']

During text preprocessing we should pay attention not to lose useful information. Smileys :-), made of punctuation, can be very important if we calculate sentiment.
We can now proceed with the <font color="#d81e5b">**calculation of prevalence**</font>, which counts the frequency of occurrence of each brand name —  <font color="#d81e5b">**subsequently standardized**</font> considering the scores of all the words in the texts. My choice of standardization here is to subtract the mean and divide by the standard deviation. Other approaches are also possible. This step is important to compare measures carried out considering different time frames or sets of documents (e.g. brand importance on Twitter in April and May). Normalization of absolute scores is necessary before summing prevalence, diversity and connectivity to obtain the Semantic Brand Score.

In [3]:
#PREVALENCE
#Import Counter and Numpy
from collections import Counter
import numpy as np

#Create a dictionary with frequency counts for each word
countPR = Counter()
for t in texts:
    countPR.update(Counter(t))

#Calculate average score and standard deviation
avgPR = np.mean(list(countPR.values()))
stdPR = np.std(list(countPR.values()))

#Calculate standardized Prevalence for each brand
PREVALENCE = {}
for brand in brands:
    PREVALENCE[brand] = (countPR[brand] - avgPR) / stdPR
    print("Prevalence", brand, PREVALENCE[brand])

Prevalence alice 6.716253083416179
Prevalence rabbit 0.23502089447171487


Next and most important step is to <font color="#d81e5b">**transform texts (list of lists of tokens) into a social network where nodes are words and links are weighted according to the number of co-occurrences between each pair of words**</font>. In this step we have to define a co-occurrence range, i.e. a maximum distance between co-occurring words (here it is set to 3). In addition, we might want to <font color="#d81e5b">**remove links which represent negligible co-occurrences**</font>, for example those of weight = 1. It can also be useful to remove isolates, if these are not brands.

In [4]:
#Import Networkx
import networkx as nx

#Choose a co-occurrence range
co_range = 5

#Create an undirected Network Graph
G = nx.Graph()

#Each word is a network node
nodes = set([item for sublist in texts for item in sublist])
G.add_nodes_from(nodes)

#Add links based on co-occurrences
for doc in texts:
    w_list = []
    length= len(doc)
    for k, w in enumerate(doc):
        #Define range, based on document length
        if (k+co_range) >= length:
            superior = length
        else:
            superior = k+co_range+1
        #Create the list of co-occurring words
        if k < length-1:
            for i in range(k+1,superior):
                linked_word = doc[i].split()
                w_list = w_list + linked_word
        #If the list is not empty, create the network links
        if w_list:    
            for p in w_list:
                if G.has_edge(w,p):
                    G[w][p]['weight'] += 1
                else:
                    G.add_edge(w, p, weight=1)
        w_list = []

#Remove negligible co-occurrences based on a filter
link_filter = 2
#Create a new Graph which has only links above
#the minimum co-occurrence threshold
G_filtered = nx.Graph() 
G_filtered.add_nodes_from(G)
for u,v,data in G.edges(data=True):
    if data['weight'] >= link_filter:
        G_filtered.add_edge(u, v, weight=data['weight'])

#Optional removal of isolates
isolates = set(nx.isolates(G_filtered))
isolates -= set(brands)
G_filtered.remove_nodes_from(isolates)

#Check the resulting graph (for small test graphs)
#G_filtered.nodes()
#G_filtered.edges(data = True)
print("Original Network\nNo. of Nodes:", G.number_of_nodes(), "No. of Edges:", G.number_of_edges())
print("Filtered Network\nNo. of Nodes:", G_filtered.number_of_nodes(), "No. of Edges:", G_filtered.number_of_edges())

Original Network
No. of Nodes: 1051 No. of Edges: 14455
Filtered Network
No. of Nodes: 691 No. of Edges: 2710


Having determined the co-occurrence network, we can now <font color="#d81e5b">**calculate diversity and connectivity**</font>, which are <font color="#d81e5b">**distinctiveness centrality**</font> (previously we used degree) and weighted betweenness centrality of a brand node. We standardize these values as we did with prevalence. More information about distinctiveness centrality is given <font color="#d81e5b">[**in this paper**](https://arxiv.org/abs/1912.03391). **You will also need to install the Python *distinctivenss* package.**</font>

In [5]:
#INSTALL AND IMPORT THE DISTINCTIVENESS PACKAGE
#pip install -U distinctiveness
from distinctiveness.dc import dc_all

#DIVERSITY
#Calculate Distinctiveness Centrality
DC = dc_all(G_filtered, normalize = False, alpha = 1)
DIVERSITY_sequence=DC["D2"]
#Calculate average score and standard deviation
avgDI = np.mean(list(DIVERSITY_sequence.values()))
stdDI = np.std(list(DIVERSITY_sequence.values()))
#Calculate standardized Diversity for each brand
DIVERSITY = {}
for brand in brands:
    DIVERSITY[brand] = (DIVERSITY_sequence[brand] - avgDI) / stdDI
    print("Diversity", brand, DIVERSITY[brand])

Diversity alice 6.016090810210237
Diversity rabbit 0.3527287505665111


If we calculate connectivity as weighted betweenness centraliy, we first have to define <font color="#d81e5b">**inverse weights**</font>, as weights are treated by Networkx as distances (which is the opposite of what we want).

In [6]:
#Define inverse weights 
for u,v,data in G_filtered.edges(data=True):
    if 'weight' in data and data['weight'] != 0:
        data['inverse'] = 1/data['weight']
    else:
        data['inverse'] = 1   

#CONNECTIVITY
CONNECTIVITY_sequence=nx.betweenness_centrality(G_filtered, normalized=False, weight ='inverse')
#Calculate average score and standard deviation
avgCO = np.mean(list(CONNECTIVITY_sequence.values()))
stdCO = np.std(list(CONNECTIVITY_sequence.values()))
#Calculate standardized Prevalence for each brand
CONNECTIVITY = {}
for brand in brands:
    CONNECTIVITY[brand] = (CONNECTIVITY_sequence[brand] - avgCO) / stdCO
    print("Connectivity", brand, CONNECTIVITY[brand])

Connectivity alice 1.1685215814353405
Connectivity rabbit 0.09234660192732538


The <font color="#d81e5b">**Semantic Brand Score**</font> of each brand is finally obtained by summing the standardized values of prevalence, diversity and connectivity. Different approaches are also possible.

In [7]:
#Obtain the Semantic Brand Score of each brand
SBS = {}
for brand in brands:
    SBS[brand] = PREVALENCE[brand] + DIVERSITY[brand] + CONNECTIVITY[brand]
    print("SBS", brand, SBS[brand])

SBS alice 13.900865475061757
SBS rabbit 0.6800962469655514


In [8]:
#Generate a final pandas data frame with all results
import pandas as pd

PREVALENCE = pd.DataFrame.from_dict(PREVALENCE, orient="index", columns = ["PREVALENCE"])
DIVERSITY = pd.DataFrame.from_dict(DIVERSITY, orient="index", columns = ["DIVERSITY"])
CONNECTIVITY = pd.DataFrame.from_dict(CONNECTIVITY, orient="index", columns = ["CONNECTIVITY"])
SBS = pd.DataFrame.from_dict(SBS, orient="index", columns = ["SBS"])

SBS = pd.concat([PREVALENCE, DIVERSITY, CONNECTIVITY, SBS], axis=1, sort=False)
SBS

,PREVALENCE,DIVERSITY,CONNECTIVITY,SBS
alice,6.716253,6.016091,1.168522,13.900865
rabbit,0.235021,0.352729,0.092347,0.680096
